In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

import numpy as np
from skimage import data
from skimage.transform import resize

import matplotlib.pyplot as plt

import torch

from src.reader import KineticDataset

from src.vqgan import ViTVQGAN
from src.my_model import MaskCode
from src.trainer import AttentionMaskModeling


In [3]:
path = "/mnt/e/kinetics-dataset/k400"
split = "train"

ds = KineticDataset(
    path, split,
    n_frames=16,
)
ds_loader = DataLoader(
    ds, 1, True, 
)

99/246534

In [6]:
is_dev = False

lightning_model = AttentionMaskModeling(
    model=MaskCode(
		window_size=(3, 3),
		length=32, height=32, width=32,
		depth=4, heads=8, dim=128, embed_dim=32,
		n_codes=8192,
	), 
    top_p=0.95
)
wandb_logger = WandbLogger(
	project="semcom",
)
# wandb_logger.experiment.config.update({
#     "dim": dim,
#     "depth": depth
# })
trainer = pl.Trainer(
    # training settings
    max_epochs=25,
    val_check_interval=1.0,
    accelerator="cpu" if is_dev else "gpu",
    precision="32-true" if is_dev else "16-mixed",
    accumulate_grad_batches=1,
    gradient_clip_val=1.0,
    # logging settings
    default_root_dir=f"./checkpoints",
    # logger=wandb_logger,
    callbacks=[
        ModelCheckpoint(
			monitor="train_acc",
			dirpath="./trained",
			filename="semcom-{epoch:02d}-{train_acc:.2f}",
			save_top_k=3,
			mode="max",
		)
	],
    # dev setting
    fast_dev_run=is_dev,
)
trainer.fit(
    lightning_model, 
    train_dataloaders=ds_loader,
    # val_dataloaders=None,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type               | Params | Mode 
-------------------------------------------------------
0 | model   | MaskCode           | 2.9 M  | train
1 | teacher | MaskCode           | 2.9 M  | train
2 | acc     | MulticlassAccuracy | 0      | train
-------------------------------------------------------
2.9 M     Trainable params
2.9 M     Non-trainable params
5.8 M     Total params
23.309    Total estimated model params size (MB)
213       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

/home/leonard/anaconda3/envs/semcom/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:209: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
`Trainer.fit` stopped: `max_epochs=25` reached.
